In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os, sys
path = os.getcwd()
while not path.endswith('research'):    
    path = os.path.abspath(os.path.join(path, '../'))
sys.path.append(path)

***

In [ ]:
#a number of gas analytics utilities:
from gmt.fom.gas.shared.utils.common import print_attributes as pa
from gmt.fom.gas.shared.utils.common import display_full_df as ddff
from gmt.fom.gas.shared.utils.common import flatten_lst

In [ ]:
#gas aligned mongo db
from gmt.fom.gas.shared.utils.mongo import GasMongo

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

***

In [ ]:
import itertools
import os, sys
import numpy as np

from datetime import datetime, date

In [ ]:
import pandas as pd

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
import sklearn as sk
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Time Series Modelling

In statistics and quantitative research methodology, a *sample* is a set of individuals or objects collected or selected from a statistical population by a defined procedure

In mathematical terms, given a probability distribution F, a random sample of length n (where n may be any positive integer) is a set of realizations of n *independent, identically distributed (iid) random variables with distribution F*.

In cross-sectional data analysis, it is commonly assumed that observations in a sample are effectively  i.i.d.

What about in time-series analysis?

For time series:
* independence is not evident at all due to temporal ordering
* identiacally distributed: is a very strong statement leading to strong stationarity (the distribution properties of blue dots and red dots (r.v.) below must not be the same)
* randomness is understood in a sence of a realized path of discrete stochastic process

*Simple stochastic process: random walk in time*

In [ ]:
N=5
paths = []
for i in range(N):
    dims = 1
    step_n = 80
    step_set = [-1, 1]
    origin = np.zeros((1,dims))
    step_shape = (step_n,dims)
    steps = np.random.choice(a=step_set, size=step_shape)
    path = np.concatenate([origin, steps]).cumsum(0)
    start = path[:1]
    stop = path[-1:]
    paths.append(path)

fig = plt.figure(figsize=(8*1.6,8))
ax = fig.add_subplot(111)

k = int(step_n/2)
l = int(step_n/3)
for i in range(len(paths)):
    if i == 2:
        ax.plot(paths[i],lw=2)
    else:
        ax.plot(paths[i],lw=1)
    ax.set_xlim(0, step_n)
    ax.plot(k, paths[i][k], c='red', marker='o')
    ax.plot(l, paths[i][l], c='blue', marker='o')
    
ax.set_xlabel('Time')
ax.set_title('Random walk in time')
plt.axvline(k, color='r')
plt.axvline(l, color='b')
plt.show()

# Data

In [ ]:
df_google = pd.read_csv('google.csv'); df_google.head()

In [ ]:
df_sp500 = pd.read_csv('sp500_index.csv'); df_sp500.head()

# Exploratory Analysis

In [ ]:
# we see that day is 'object' - we need to convert to datetime
df_google.dtypes.to_frame();

**`pd.to_datetime()`**

In [ ]:
df_google['day'] = pd.to_datetime(df_google['day'])

df_google.head(1)

In [ ]:
df_sp500['day'] = pd.to_datetime(df_sp500['day'])

df_sp500.head(1)

**`describe()`**

In [ ]:
df_google.describe()

**let's set day column to be index of the dataFrame**

In [ ]:
df_google.set_index('day', inplace=True)
df_sp500.set_index('day', inplace=True)

**plots**

In [ ]:
fig, ax = plt.subplots()
df_google['close'].plot(figsize=(8*1.6,8), ax=ax, label='google')
idx = df_google.index
df_sp500.loc[idx]['close'].plot(figsize=(8*1.6,8), ax=ax, label='sp500')
ax.set_title('GOOGLE and SP500 prices')
ax.legend()
plt.show()

# Data preprocessing and stationarity

Trends can result in a varying mean over time, whereas seasonality can result in a changing variance over time, both which define a time series as being non-stationary.

trend:  
- retression on time
- subtract trend --> work with residuals $\hat{y}_t = x_t - \hat{\mu_t}$, where $\mu_t$ - trend
- include lagged variables



seasonlaty:
* time dummy variables
* difference
    

transformations:
* $y_t = \log x_t$ : tends to supress larger fluctuations that occur over portions of the series where the underlying values are larger
* Box-Cox family:
$$
 y_t = \begin{cases} 
     (x_t^\lambda -1) / \lambda, & \text{$\lambda \neq 0$} \\
     \log x_t, & \text{$\lambda =0$ } 
 \end{cases}
$$
* domain based

In [ ]:
google_close = df_google['close']

In [ ]:
df_google['close'].plot(figsize=(8*1.6,8), title='Google Stock Price')

In [ ]:
decompose_result = sm.tsa.seasonal_decompose(google_close.resample('1W').mean().dropna())
with mpl.rc_context():
    mpl.rc("figure", figsize=(9*1.6,9))
    trend = decompose_result.trend
    seasonal = decompose_result.seasonal
    residual = decompose_result.resid
    decompose_result.plot();

*The time series is clearly `non-stationary`, so we need to transform to obtain (weak) stationarity.* 

We will be using `log-returns`:
$$
r_t = \log\frac{X_t}{X_{t-1}}
$$

In [ ]:
google_rt = np.log(google_close.sort_index(ascending=True).divide(google_close.sort_index(ascending=True).shift(1))).sort_index(ascending=False)
google_rt = google_rt[~google_rt.isnull()]

In [ ]:
pd.DataFrame(pd.concat([google_close, google_rt], axis=1).reset_index().values, columns=['day', 'close', 'r_t']).set_index('day').sort_index(ascending=False).head()

In [ ]:
np.log(1740.05/1726.13)

In [ ]:
fig, ax = plt.subplots()
google_rt.plot(figsize=(8*1.6,8), title='Google Log Return', ax=ax)
x_vals = ax.get_xticks()
plt.axhline(0.0, color='r')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sm.graphics.qqplot(google_rt, line='q', fit=True, ax=ax)
ax.set_title('QQ Plot of Google Log Return')
plt.show()

**Augmented Dickey-Fuller test**: $H_0$: a unit root is present in a time series sample $\Rightarrow$ non-stationary.

In [ ]:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
adf_test(google_rt)

**We see `p-value` is less then both 0.05 and 0.01 significance levels* we `reject` $H_0$ of a presence of unit root in our time series!

# Training | (Validation) |  Testing splits

In [ ]:
tscv = TimeSeriesSplit(n_splits=9)

In [ ]:
X = google_rt[:300].sort_index(ascending=True).dropna()

In [ ]:
X = X.resample('1D').mean().dropna()

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(9*1.6,9))
i=j=0
res=[]
for train_index, test_index in tscv.split(X):      
    X_train, X_test = X[train_index].resample('1D').mean(), X[test_index].resample('1D').mean()
    X_train.dropna().plot(ax=axs[i,j], label='train')
    X_test.dropna().plot(ax=axs[i,j], label='test')
    res.append((X_train,X_test))
    idx  = pd.concat([X_train, X_test], axis=0).index
    axs[i,j].set_xticks(idx[0::(j+3)*10])
    axs[i,j].legend()        
    j+=1
    if j%3==0:
        i+=1
        j=0
plt.tight_layout()
plt.show()

# Modelling

At this moment we have generated 9 train/test splits. 

Below we will demonstrate the process of 
* calibrating the model
* forecasting
* computing performance metrics

for just `the last` (out of 9) X_train, X_test pairs.

Subsequently, this process will be repeteaded 9 times and the average of the performance metrics will become a `cross-validation error.`

**Sample Autocorrelation | Partial autocorrelation functions**

**ACF** is an (complete) auto-correlation function which gives us values of auto-correlation of any series with its lagged values. We plot these values along with the confidence. 

In simple terms, it describes how well the present value of the series is related with its past values. A time series can have components like trend, seasonality, cyclic and residual. ACF considers all these components while finding correlations hence it’s a ‘complete auto-correlation plot’.

**PACF** is a partial auto-correlation function. Basically instead of finding correlations of present with lags like ACF, it finds correlation of the residuals (which remains after removing the effects which are already explained by the earlier lag(s)) with the next lag value hence ‘partial’ and not ‘complete’ as we remove already found variations before we find the next correlation. 

So if there is any hidden information in the residual which can be modeled by the next lag, we might get a good correlation and we will keep that next lag as a feature while modeling. Remember while modeling we don’t want to keep too many features which are correlated as that can create multicollinearity issues. Hence we need to retain only the relevant features.

In [ ]:
fig = plt.figure(figsize=(9*1.6,9))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(X_train.dropna(), lags=20, ax=ax1)
ax1.set_ylim(-0.5,0.5)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(X_train.dropna(), lags=20, ax=ax2, method='ols')
ax2.set_ylim(-0.5,0.5)
plt.show()

## AR(I)MA

**AR(p)** 
$$
y_t = c + \phi_1y_{t-1} + \phi_2y_{t-2}+\ldots+\phi_py_{t-p}+\varepsilon_t
$$

Here $\varepsilon_t$ is white noise (a collection of uncorrelated random variables, with mean $0$ and finite variance $\sigma^2$)

Order **p** is the lag value after which *PACF* plot crosses the upper confidence interval for the first time. 

These p lags will act as our features while forecasting the AR time series. We cannot use the ACF plot here because it will show good correlations even for the lags which are far in the past. If we consider those many features, we will have multicollinearity issues.This is not a problem with PACF plot as it removes components already explained by earlier lags, so we only get the lags which have the correlation with the residual i.e the component not explained by earlier lags.

**MA(q)**
$$
y_t = c + \varepsilon_t + \theta_1\varepsilon_{t-1}+\theta_2\varepsilon_{t-2}+\ldots+\theta_q\varepsilon_{t-q}
$$

Here $\varepsilon_t$ is white noise (a collection of uncorrelated random variables, with mean $0$ and finite variance $\sigma^2$)

Order **q** of the MA process is obtained from the *ACF* plot, this is the lag after which ACF crosses the upper confidence interval for the first time. 

As we know PACF captures correlations of residuals and the time series lags, we might get good correlations for nearest lags as well as for past lags. Why would that be? Since our series is linear combination of the residuals and none of time series own lag can directly explain its present (since its not an AR), which is the essence of PACF plot as it subtracts variations already explained by earlier lags, its kind of PACF losing its power here! On the other hand being a MA process, it doesn’t have the seasonal or trend components so the ACF plot will capture the correlations due residual components only. You can also think of it as ACF which is a complete plot (capturing trend, seasonality, cyclic and residual correlations) acting as a partial plot since we don’t have trends, seasons, etc.

**Ljung–Box** test $H_0$: The data are independently distributed (i.e. the correlations in the population from which the sample is taken are 0, so that any observed correlations in the data result from randomness of the sampling process)

**Jarque–Bera** test $H_0$: is a goodness-of-fit test of whether sample data have the skewness and kurtosis matching a normal distribution. A joint $H_0$ of the skewness being zero and the excess kurtosis being zero.

In [1]:
mod = sm.tsa.arima.ARIMA(endog=X_train, order=(2,0,2), trend='n', enforce_stationarity=True, freq='1D')
res = mod.fit(cov_type='robust')

# we concatenate the last known price with the forecasted returns to get the following list:

forecast = pd.concat([
    pd.Series(google_close[X_train.index.max()], index=[X_train.index.max()]),
    res.predict(X_test.resample('1D').mean().index.min(), X_test.resample('1D').mean().index.max())
], axis=0)
forecast[:4]

# now using the log_return lambda function

log_return = lambda a,b: a*np.exp(b)

# we cacluate the multi-period stock price forecast

idx = X_test.index.append(pd.DatetimeIndex([X_train.index.max()])).sort_values()
forecast = pd.DataFrame([*itertools.accumulate(forecast.values,log_return)], index=idx, columns=['forecast']); forecast.head()

performance_df = forecast.join(google_close).dropna()

mae = mean_absolute_error(performance_df.close.values, performance_df.forecast.values)

mse = mean_squared_error(performance_df.close.values, performance_df.forecast.values)

mpe = mean_percentage_error(performance_df.close.values, performance_df.forecast.values)

mape = mean_absolute_percentage_error(performance_df.close.values, performance_df.forecast.values)

print(f'Performance metrics summary:\nMAE = {mae:.2f}\nMSE = {mse:.2f}\nMPE = {mpe:.2f}\nMAPE = {mape:.2f}')

NameError: name 'sm' is not defined

# Cross Validation Error 

**Now repeating the above for all 9 splits we can compute the Cross-Validation error, choosing one particular metric as representative one: `MAPE`**